In [1]:
import pandas as pd
from io import StringIO
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

In [2]:
import csv
import seaborn as sns

In [3]:
def get_data():
    df = pd.read_csv('mySddf.csv')
    return df

In [4]:
lp=pd.read_csv('mySddf.csv')

lp.isnull().sum()
list_drop=['disease']
lp.drop(list_drop,axis=1, inplace= True)
lp.head()

,Unnamed: 0,Disease,Symptom_1,Symptom_2,Symptom_3,Description,Precaution_1,Precaution_2,Precaution_3,Precaution_4
0,0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,"In humans, fungal infections occur when an inv...",bath twice,use detol or neem in bathing water,keep infected area dry,use clean cloths
1,1,Allergy,continuous_sneezing,shivering,chills,An allergy is an immune system response to a f...,apply calamine,cover area with bandage,NaN,use ice to compress itching
2,2,GERD,stomach_pain,acidity,ulcers_on_tongue,"Gastroesophageal reflux disease, or GERD, is a...",avoid fatty spicy food,avoid lying down after eating,maintain healthy weight,exercise
3,3,Chronic cholestasis,itching,vomiting,yellowish_skin,"Chronic cholestatic diseases, whether occurrin...",cold baths,anti itch medicine,consult doctor,eat healthy
4,4,Drug Reaction,itching,skin_rash,stomach_pain,An adverse drug reaction (ADR) is an injury ca...,stop irritation,consult nearest hospital,stop taking drug,follow up


In [5]:
def data_prepare():# all this work for creating dictionary
    col = ['Unnamed: 0','Disease','Symptom_1','Symptom_2','Symptom_3', 'Description', 'Precaution_1', 'Precaution_2', 'Precaution_3', 'Precaution_4']
    y= get_data() #provides access to file resources distributed with the package.
    y = y[col]
    y = y[pd.notnull(y['Description'])] # removes the row if found null
    y.columns = ['Unnamed: 0','Disease','Symptom_1','Symptom_2','Symptom_3', 'Description', 'Precaution_1', 'Precaution_2', 'Precaution_3', 'Precaution_4']
    y['category_id'] = y['Disease'].factorize()[0]
    #print(y['category_id'])
    category_id_df = y[['Disease', 'category_id']].drop_duplicates().sort_values('category_id')#removes duplicates
    category_to_id = dict(category_id_df.values)
    id_to_category = dict(category_id_df[['Disease','category_id']].values)#This will add the column in our dataframe
    return y


I have used the Multinomial Naive_Bayes algorithm for prediction because I find it easy to implement and it has high accuracy.
The OneVsRest strategy can be used for multi-label learning, where a classifier is used to predict multiple labels for instance. Naive Bayes supports multi-class, but we are in a multi-label scenario, therefore, we wrap Naive Bayes in the OneVsRestClassifier.

OneVsRest multi-label strategy:
The Multi-label algorithm accepts a binary mask over multiple labels. The result for each prediction will be an array of 0s and 1s marking which class labels apply to each row input sample.


In [6]:
def naive_algo(choice):
    tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
    df=data_prepare()
    features = tfidf.fit_transform(df.Disease).toarray()
    labels = df.category_id
    features.shape
    X_train, X_test, y_train, y_test = train_test_split(df['Disease'], df[choice], random_state = 0)
    count_vect = CountVectorizer()
    X_train_counts = count_vect.fit_transform(X_train)
    tfidf_transformer = TfidfTransformer()
    X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
    clf = MultinomialNB().fit(X_train_tfidf, y_train)
    return clf,count_vect

In [7]:
def predict(Disease,choice):
    clf,count_vect=naive_algo(choice)
    intent=clf.predict(count_vect.transform([Disease]))
    intent=str(intent).strip("['']")
    print(intent)
    return intent

In [11]:
ques=input("Enter your disease ")

Enter your disease AIDS


In [14]:
for i in range(8):
    if(i==0):
        print("Symptoms are: ")
        x=predict(ques,'Symptom_1')
        intent=str(x).strip("['']")
    elif(i==1):
        x=predict(ques,'Symptom_2')
        intent=str(x).strip("['']")
    elif(i==2):
        x=predict(ques,'Symptom_3')
        intent=str(x).strip("['']")
        print('\n')
    elif(i==3):
        print("Description:")
        x=predict(ques,'Description')
        intent=str(x).strip("['']")
        print('\n')
    elif(i==4):
        print("Precautions")
        x=predict(ques,'Precaution_1')
        intent=str(x).strip("['']")
    elif(i==5):
        x=predict(ques,'Precaution_2')
        intent=str(x).strip("['']")

Symptoms are: 
itching
 vomiting
 high_fever


Description:
"Acquired immunodeficiency syndrome (AIDS) is a chronic, potentially life-threatening condition caused by the human immunodeficiency virus (HIV). By damaging your immune system, HIV interferes with your body's ability to fight infection and disease."


Precautions
Consult nearest hospital
wear ppe if possible


In [ ]:
X_train